In [1]:
from __future__ import absolute_import, division, print_function

import sys
import os
import time

sys.path.insert(0,'../../')
sys.path.insert(0,'../../py')

import parameters
import utilities
import spectrogram_utilities
import output_utilities
import spectrogram_output_visualiser
import spectrogram_cuts_db_creation

import tensorflow as tf
import numpy as np
import pandas as pd
import h5py


from scipy import signal
from sklearn import preprocessing
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# matplotlib.get_backend()
%matplotlib inline
#
tf.enable_eager_execution() 
tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
# constants
awd_event = 1
site = parameters.sites[0]

## Database

### Loading database

In [6]:
data, probs, labels, f_cut_length, t_cut_length = spectrogram_cuts_db_creation.load_spectrogram_cuts_db(awd_event, site, database_name='training_cuts.h5', verbose=True)


Loading spectrogram cuts from database for awdEvent1/marion
0%.........10%.........20%.........30%.........40%.........50%.........60%.........70%.........80%.........90%.........
Runtime: 25.47 seconds


## Convolutional Neural Network

#### Create TFRecords

In [30]:
def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))
def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def createTFRecord(tfrecord_name, data, labels):
    # check for data size
    assert data.shape[0]==labels.shape[0]
    # open TRFrecords file
    writer = tf.python_io.TFRecordWriter(tfrecord_name)
    i = 0
    for sample, label in zip(data,labels):
        percent = int(num_file*100/len(files))
            if last_percent != percent:
                if percent%10==0:
                    sys.stdout.write("%s%%" % percent)
                    sys.stdout.flush()
                else:
                    sys.stdout.write(".")
                    sys.stdout.flush()
                last_percent = percent
            num_file+=1
        # create a feature
        feature = {
            'data': _float_feature(sample.astype(np.float64)),
            'label': _int64_feature(label)
        }
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        # serialize to string and write on the file
        writer.write(example.SerializeToString())
    writer.close()
    sys.stdout.flush()
    print('Done')

In [31]:
type([data[0]])

list

In [32]:
createTFRecord('training_cuts.tfrecord',data,labels)

Train data: 0/24028
Train data: 2402/24028


KeyboardInterrupt: 

In [5]:
assert features.shape[0]==labels.shape[0]
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).shuffle(1000).batch(32)


In [6]:
assert features.shape[0]==labels.shape[0]

features_placeholder = tf.placeholder(features.dtype, features.shape)
labels_placeholder = tf.placeholder(labels.dtype, labels.shape)
dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))


In [ ]:
input_layer = tf.reshape(data,[-1, f_cut_length, t_cut_length, 1])

In [8]:
def cnn_model_fn(features, labels, mode):
    '''Model function for CNN.'''
    
    # input layer
    input_layer = tf.reshape(features,[-1, f_cut_length, t_cut_length, 1])
    
    # convolutional layer #1
    conv1 = tf.layers.conv2d(
                inputs=input_layer,
                filters=8,
                kernel_size=[5,5],
                padding='same',
                activation=tf.nn.relu)
    
    # pooling layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2)
    
    # convolutional layer #2 and pooling layer #2
    conv2 = tf.layers.conv2d(
                inputs=pool1,
                filters=16,
                kernel_size=[5,5],
                padding='same',
                activaltion=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2)
    
    # denser layer
    pool2_flat = tf.reshape(pool2, [-1, 7*7*64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # logit layer
    logits = tf.layers.dense(inputs=dropout, units=2)
    
    predictions = {
        # generate predictions for (PREDICT and EVAL mode)
        'classes': tf.argmax(input=logits, axis=1),
        # add 'softmax_tensor' to the graph. It is used for PREDICT and by the 'looging_hook'
        'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
    }
    
    if mode==tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    # configure the training optimizer (for TRAIN mode)
    if mode==tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        'accuracy': tf.metrics.accuracy(labels=labels, predictions=predictions['classes'])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    

In [ ]:

data = preprocessing.scale(data)

start = time.time()
end = time.time()
print("\nRuntime: {:.2f} seconds".format(end - start))
